# Your name: 

This notebook implements timeloop/accelergy-based energy estimation for the neural network model you trained. This part has to be run with the docker we provide, and does not require GPU support. 

One strategy to reduce the profiling time is to design a model with repeated layers since layers with the same architecture only need one time of profiling.
The profiler will also automatically save the information of profiled layers to a .json file specifiled by `profiled_lib_dir`, so that next time the same layer is profiled, the results can be obtained immediately. 


### 1. Load the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Change this model class to the architecture you used
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

### 2. Run the Profiler to estimate the peak activation size

In [ ]:
from profiler import count_activation_size
peak_activation_size = count_activation_size(
    net=net,
    input_size=(1, 3, 32, 32),
)

print(f"Peak Activation Sizes: {peak_activation_size} Byte")

### 3. Run the Profiler for Timeloop/Accelergy

In [ ]:
from profiler import Profiler
from datetime import date

today = date.today()
sub_dir = "network-" + today.strftime("%b-%d-%Y")

profiler = Profiler(
    top_dir='workloads',
    sub_dir=sub_dir,
    timeloop_dir='simple_weight_stationary',
    model=net,
    input_size=(3, 32, 32),
    batch_size=1,
    convert_fc=True,
    exception_module_names=[],
    profiled_lib_dir=f"profiled_lib.json"
)

layer_wise, overall = profiler.profile()

for layer_id, info in layer_wise.items():
    print(f"Name: {info['name']} \t Energy: {info['energy']:.2f} \t Cycle: {info['cycle']} \t Number of same architecture layers: {info['num']}")
    
print(f"\nTotal Energy: {overall['total_energy']/1e9:.8f} mj \nTotal Cycles: {overall['total_cycle']/1e6:.8f} Million")

print(f"MACs: {overall['macs']}\nNum of Parameters: {overall['num_params']} \nPeak Activation Size: {overall['activation_size']} Byte")


